In [ ]:
# 첫 번째 셀
import sys
sys.path.append('../core')

from mcp_client import MCPClient
import asyncio
import json

# 두 번째 셀 - 포트 수정하여 연결 테스트
async def test_connection():
    # 서버가 8000 포트에서 실행 중이므로 포트 수정
    client = MCPClient(server_url="http://localhost:8000")
    try:
        success = await client.connect()
        if success:
            print("✅ 연결 성공!")
            info = await client.get_server_info()
            print("서버 정보:")
            print(json.dumps(info, ensure_ascii=False, indent=2))
        else:
            print("❌ 연결 실패")
    finally:
        await client.disconnect()

# 실행
await test_connection()

In [ ]:
# test_client.py에 추가하거나 새로운 함수로 테스트
async def test_tool_call():
    client = MCPClient(server_url="http://localhost:8000")
    try:
        await client.connect()
        print("🔧 도구 호출 테스트...")
        result = await client.call_tool("get_env_info")
        print("도구 호출 결과:")
        print(json.dumps(result, ensure_ascii=False, indent=2))
        return result
    finally:
        await client.disconnect()


await test_tool_call()

In [9]:
# 첫 번째 셀
import os
import sys
sys.path.append('../core')

from langchain_agent import LangChainAgent
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

# 두 번째 셀 - Bitget 도구 테스트
async def test_bitget_tools():
    agent_client = LangChainAgent(model_key="gpt41")            
    mcp_server_url = f"http://{os.getenv('MCP_SERVER_HOST', 'localhost')}:{os.getenv('MCP_SERVER_PORT', '8000')}/mcp"
    
    async with streamablehttp_client(mcp_server_url) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # 에이전트 생성
            agent = await agent_client.create_agent(session, "bitget_agent")
            if not agent:
                print("❌ 에이전트 생성 실패")
                return
            
            print("✅ Bitget 도구 포함 에이전트 생성 완료")
            
            # Bitget 관련 질문들
            questions = [
                #"비트코인에 대한 다양한 시황분석통해서 레포트 작성해줘",
                "지난 7일간 바이낸스와 코인베이스의 자금 흐름을 분석해줘"
                #"Reddit에서 암호화폐 커뮤니티 심리를 분석해줘", 
                #"고래들의 최근 거래 활동과 거래소 흐름을 확인해줘",
                #"현재 시장 상황을 온체인 데이터와 소셜 데이터로 종합 분석해줘"
            ]
            
            for question in questions:
                print(f"\n👤 질문: {question}")
                answer = await agent_client.ask_agent(question, "bitget_agent")
                print(f"🤖 답변: {answer}")

# 실행
await test_bitget_tools()

Invalid key for model 'gpt41'. Set model gpt4mini as default.
  - get_bitget_account_info: 
        Get Bitget futures account information in...
  - get_bitget_positions: 
        Get all current positions in Bitget futur...
  - get_bitget_price: 
        Get current price information for a speci...
  - get_bitget_candles: 
        Get candlestick data for a specific symbo...
  - get_bitget_open_orders: 
        Get all open orders for a specific symbol...
  - place_bitget_order: 
        Place a futures order on Bitget.
        ...
  - cancel_bitget_order: 
        Cancel a futures order on Bitget.
       ...
  - set_bitget_leverage: 
        Set leverage for a specific symbol.
     ...
  - get_bitget_order_history: 
        Get order history for a specific symbol o...
  - close_all_bitget_positions_correct: 
        Close all open positions using correct Bi...
  - close_bitget_position_correct: 
        Close specific position using correct Bit...
  - show_position_details: 
        

In [ ]:
# SSE 응답 파싱을 수정한 MCP 클라이언트 테스트
import sys
import os
import asyncio
import json
sys.path.append('../core')

async def test_fixed_mcp_tools():
    """SSE 응답을 제대로 파싱하여 MCP 도구 테스트"""
    print("🔧 수정된 MCP 클라이언트 테스트")
    print("="*60)
    
    import aiohttp
    
    server_url = "http://localhost:8000"
    
    def parse_sse_response(sse_text):
        """Server-Sent Events 응답 파싱"""
        lines = sse_text.strip().split('\n')
        data_lines = []
        
        for line in lines:
            if line.startswith('data: '):
                data_content = line[6:]  # 'data: ' 제거
                data_lines.append(data_content)
        
        if data_lines:
            # 여러 data 라인이 있을 수 있으므로 합치기
            full_data = '\n'.join(data_lines)
            try:
                return json.loads(full_data)
            except json.JSONDecodeError:
                return None
        return None
    
    async def call_mcp_tool(tool_name, params=None):
        """MCP 도구 호출 (SSE 파싱 수정)"""
        if params is None:
            params = {}
        
        payload = {
            "jsonrpc": "2.0",
            "id": f"test_{tool_name}",
            "method": "tools/call",
            "params": {
                "name": tool_name,
                "arguments": params
            }
        }
        
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json, text/event-stream",
            "Cache-Control": "no-cache"
        }
        
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post(
                    f"{server_url}/mcp",
                    json=payload,
                    headers=headers
                ) as response:
                    
                    if response.status == 200:
                        raw_text = await response.text()
                        
                        # SSE 응답 파싱
                        parsed_response = parse_sse_response(raw_text)
                        
                        if parsed_response and 'result' in parsed_response:
                            result = parsed_response['result']
                            
                            # MCP 표준 응답 구조 처리
                            if isinstance(result, dict) and 'content' in result:
                                content = result['content']
                                if isinstance(content, list) and len(content) > 0:
                                    # 첫 번째 content 항목의 text 추출
                                    first_content = content[0]
                                    if isinstance(first_content, dict) and 'text' in first_content:
                                        actual_result = first_content['text']
                                        
                                        # JSON 문자열인 경우 다시 파싱
                                        try:
                                            return json.loads(actual_result)
                                        except json.JSONDecodeError:
                                            return actual_result
                            
                            return result
                        elif parsed_response and 'error' in parsed_response:
                            return {'error': parsed_response['error']}
                        else:
                            return {'error': 'Invalid response format'}
                    else:
                        return {'error': f'HTTP {response.status}'}
                        
        except Exception as e:
            return {'error': str(e)}
    
    # 테스트할 도구들
    test_cases = [
        {
            'name': 'get_bitget_price',
            'params': {'symbol': 'BTCUSDT'},
            'description': 'Bitget BTC 가격 조회'
        },
        {
            'name': 'get_latest_crypto_news',
            'params': {'sources': ['cryptocompare'], 'limit_per_source': 3},
            'description': '최신 암호화폐 뉴스'
        },
        {
            'name': 'analyze_rsi',
            'params': {'symbol': 'BTCUSDT', 'period': 14, 'interval': '1h'},
            'description': 'BTC RSI 분석'
        },
        {
            'name': 'analyze_moving_averages',
            'params': {'symbol': 'BTCUSDT', 'short_ma': 20, 'long_ma': 50, 'interval': '1h'},
            'description': 'BTC 이동평균 분석'
        },
        {
            'name': 'comprehensive_technical_analysis',
            'params': {'symbol': 'BTCUSDT', 'interval': '1h'},
            'description': 'BTC 종합 기술분석'
        },
        {
            'name': 'get_trending_crypto_info',
            'params': {},
            'description': '트렌딩 암호화폐 정보'
        }
    ]
    
    results = {}
    
    for test_case in test_cases:
        tool_name = test_case['name']
        params = test_case['params']
        description = test_case['description']
        
        print(f"\n🔍 [{description}] {tool_name} 테스트:")
        print(f"   파라미터: {params}")
        
        try:
            start_time = asyncio.get_event_loop().time()
            result = await call_mcp_tool(tool_name, params)
            end_time = asyncio.get_event_loop().time()
            
            response_time = end_time - start_time
            
            if isinstance(result, dict):
                if 'error' in result:
                    print(f"   ❌ 오류: {result['error']}")
                    status = "FAILED"
                else:
                    print(f"   ✅ 성공! (응답시간: {response_time:.2f}초)")
                    
                    # 결과 내용 요약
                    if tool_name == 'get_bitget_price':
                        if 'data' in result and 'markPrice' in result['data']:
                            price = result['data']['markPrice']
                            print(f"   💰 BTC 가격: ${price}")
                        else:
                            print(f"   📊 응답 키: {list(result.keys())}")
                    
                    elif tool_name == 'get_latest_crypto_news':
                        if 'summary' in result:
                            summary = result['summary']
                            total_news = summary.get('total_news', 0)
                            print(f"   📰 총 뉴스: {total_news}개")
                            
                            if 'sentiment_distribution' in summary:
                                sentiment = summary['sentiment_distribution']
                                print(f"   😊 감정분석: 긍정 {sentiment.get('positive', 0)}, 부정 {sentiment.get('negative', 0)}, 중립 {sentiment.get('neutral', 0)}")
                        else:
                            print(f"   📊 응답 키: {list(result.keys())}")
                    
                    elif tool_name == 'analyze_rsi':
                        if 'current_rsi' in result:
                            rsi = result['current_rsi']
                            signal = result.get('signal', 'N/A')
                            print(f"   📈 현재 RSI: {rsi} ({signal})")
                        else:
                            print(f"   📊 응답 키: {list(result.keys())}")
                    
                    elif tool_name == 'analyze_moving_averages':
                        if 'moving_averages' in result:
                            ma_data = result['moving_averages']
                            ma20 = ma_data.get('ma_20', 'N/A')
                            ma50 = ma_data.get('ma_50', 'N/A')
                            cross_signal = result.get('cross_signal', 'N/A')
                            print(f"   📊 MA20: ${ma20}, MA50: ${ma50}")
                            print(f"   🎯 신호: {cross_signal}")
                        else:
                            print(f"   📊 응답 키: {list(result.keys())}")
                    
                    elif tool_name == 'comprehensive_technical_analysis':
                        if 'overall_signal' in result:
                            signal = result['overall_signal']
                            confidence = result.get('confidence_score', 'N/A')
                            recommendation = result.get('recommendation', 'N/A')
                            print(f"   🎯 종합신호: {signal} (신뢰도: {confidence})")
                            print(f"   💡 추천: {recommendation[:50]}...")
                        else:
                            print(f"   📊 응답 키: {list(result.keys())}")
                    
                    elif tool_name == 'get_trending_crypto_info':
                        if 'trending_analysis' in result:
                            trending = result['trending_analysis']
                            if 'trending_coins' in trending:
                                coins = trending['trending_coins']
                                print(f"   🔥 트렌딩 코인 수: {len(coins)}개")
                                if coins:
                                    print(f"   🥇 1위: {coins[0].get('name', 'N/A')}")
                        else:
                            print(f"   📊 응답 키: {list(result.keys())}")
                    
                    status = "SUCCESS"
                    
            else:
                print(f"   ⚠️ 예상과 다른 응답 타입: {type(result)}")
                print(f"   📄 응답 내용: {str(result)[:100]}...")
                status = "PARTIAL"
            
            results[tool_name] = {
                'status': status,
                'response_time': response_time,
                'result': result
            }
            
        except Exception as e:
            print(f"   ❌ 예외 발생: {e}")
            results[tool_name] = {
                'status': 'ERROR',
                'response_time': 0,
                'error': str(e)
            }
    
    # 결과 요약
    print(f"\n" + "="*60)
    print("📊 테스트 결과 요약")
    print("="*60)
    
    success_count = len([r for r in results.values() if r['status'] == 'SUCCESS'])
    total_count = len(results)
    avg_time = sum(r.get('response_time', 0) for r in results.values()) / total_count
    
    print(f"✅ 성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"⏱️ 평균 응답시간: {avg_time:.2f}초")
    
    print(f"\n📋 개별 결과:")
    for tool_name, result in results.items():
        status_icon = {
            'SUCCESS': '✅',
            'FAILED': '❌', 
            'PARTIAL': '⚠️',
            'ERROR': '💥'
        }.get(result['status'], '❓')
        
        print(f"  {status_icon} {tool_name}: {result['status']} ({result.get('response_time', 0):.2f}초)")
    
    if success_count >= total_count * 0.8:
        print(f"\n🎉 우수한 성능! 대부분의 도구가 정상 작동합니다.")
    elif success_count >= total_count * 0.5:
        print(f"\n👍 양호한 성능! 절반 이상의 도구가 작동합니다.")
    else:
        print(f"\n⚠️ 개선 필요! 도구 설정을 점검해주세요.")
    
    print(f"\n💡 이제 AI 에이전트도 정상 작동할 것입니다!")
    
    return results

# 테스트 실행
test_results = await test_fixed_mcp_tools()

In [12]:
# 상세 데이터 검증 및 고래 거래 디버깅 코드
import json
import requests
import asyncio
import os
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Any
import aiohttp
from collections import defaultdict, Counter
import pandas as pd

# API 키 설정
ETHERSCAN_API_KEY = "9MW3Q8M8ZBR9CUA28MXXHA4Y8XZ6KPWMN5"

print("🔍 상세 데이터 검증 및 고래 거래 디버깅")
print("=" * 60)

async def make_api_request(url: str, timeout: int = 15) -> Optional[Dict]:
    """비동기 API 요청"""
    try:
        async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=timeout)) as session:
            async with session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    return data
                else:
                    print(f"❌ API 요청 실패: {response.status} - {url}")
                    return None
    except Exception as e:
        print(f"❌ API 요청 오류: {e}")
        return None

def wei_to_ether(wei_value: str) -> float:
    """Wei를 Ether로 변환"""
    try:
        if isinstance(wei_value, str):
            if wei_value.startswith('0x'):
                return int(wei_value, 16) / 1e18
            else:
                return int(wei_value) / 1e18
        return float(wei_value) / 1e18
    except:
        return 0.0

async def get_current_crypto_price(symbol: str = "ethereum") -> float:
    """현재 암호화폐 가격 가져오기"""
    try:
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={symbol}&vs_currencies=usd"
        price_data = await make_api_request(url)
        if price_data and symbol in price_data:
            return price_data[symbol]['usd']
        return 2500.0
    except Exception as e:
        print(f"❌ 가격 조회 실패: {e}")
        return 2500.0

# 테스트 1: 실제 블록 데이터 상세 검증
async def test_detailed_block_data():
    """실제 블록 데이터 상세 검증"""
    print("\n🔍 테스트 1: 실제 블록 데이터 상세 검증")
    print("-" * 50)
    
    try:
        # 최신 블록 번호 가져오기
        url = f"https://api.etherscan.io/api?module=proxy&action=eth_blockNumber&apikey={ETHERSCAN_API_KEY}"
        response = await make_api_request(url)
        latest_block = int(response['result'], 16)
        
        print(f"📊 최신 블록: {latest_block:,}")
        
        # 최신 블록 상세 정보
        block_url = f"https://api.etherscan.io/api?module=proxy&action=eth_getBlockByNumber&tag=0x{latest_block:x}&boolean=true&apikey={ETHERSCAN_API_KEY}"
        block_data = await make_api_request(block_url)
        
        if block_data and 'result' in block_data:
            block = block_data['result']
            
            print(f"\n📦 블록 {latest_block} 상세 정보:")
            print(f"   🕐 타임스탬프: {block.get('timestamp', 'N/A')}")
            print(f"   📊 거래 수: {len(block.get('transactions', []))}")
            print(f"   ⛽ Gas 사용: {block.get('gasUsed', 'N/A')}")
            print(f"   ⛽ Gas 한도: {block.get('gasLimit', 'N/A')}")
            print(f"   🔗 해시: {block.get('hash', 'N/A')}")
            
            # 거래 데이터 샘플 검증
            transactions = block.get('transactions', [])
            if transactions:
                print(f"\n🔍 거래 데이터 샘플 검증 (처음 5개):")
                
                for i, tx in enumerate(transactions[:5]):
                    print(f"\n   거래 {i+1}:")
                    print(f"      🔗 해시: {tx.get('hash', 'N/A')}")
                    print(f"      📤 From: {tx.get('from', 'N/A')}")
                    print(f"      📥 To: {tx.get('to', 'N/A')}")
                    print(f"      💰 Value (Wei): {tx.get('value', 'N/A')}")
                    
                    # Wei → ETH → USD 변환 테스트
                    value_wei = tx.get('value', '0x0')
                    if value_wei != '0x0':
                        value_eth = wei_to_ether(value_wei)
                        eth_price = await get_current_crypto_price()
                        value_usd = value_eth * eth_price
                        
                        print(f"      💰 Value (ETH): {value_eth:.6f}")
                        print(f"      💰 Value (USD): ${value_usd:.2f}")
                        
                        if value_usd > 1000:
                            print(f"      🎯 이것은 1000달러 이상 거래입니다!")
                    else:
                        print(f"      💰 Value: $0 (토큰 전송 또는 컨트랙트 호출)")
                
                # 값이 있는 거래만 필터링
                value_transactions = []
                for tx in transactions:
                    value_wei = tx.get('value', '0x0')
                    if value_wei != '0x0':
                        value_eth = wei_to_ether(value_wei)
                        if value_eth > 0:
                            value_transactions.append({
                                'hash': tx.get('hash'),
                                'value_eth': value_eth,
                                'value_usd': value_eth * await get_current_crypto_price()
                            })
                
                print(f"\n📊 이 블록의 ETH 전송 거래 통계:")
                print(f"   총 거래: {len(transactions)}")
                print(f"   ETH 전송 거래: {len(value_transactions)}")
                
                if value_transactions:
                    max_tx = max(value_transactions, key=lambda x: x['value_usd'])
                    total_eth = sum(tx['value_eth'] for tx in value_transactions)
                    total_usd = sum(tx['value_usd'] for tx in value_transactions)
                    
                    print(f"   최대 거래: ${max_tx['value_usd']:.2f} ({max_tx['value_eth']:.4f} ETH)")
                    print(f"   총 ETH 이동: {total_eth:.4f} ETH")
                    print(f"   총 USD 이동: ${total_usd:.2f}")
                    
                    # 1만 달러 이상 거래 찾기
                    large_txs = [tx for tx in value_transactions if tx['value_usd'] >= 10000]
                    print(f"   1만 달러 이상 거래: {len(large_txs)}개")
                    
                    if large_txs:
                        print(f"   🐋 발견된 대형 거래들:")
                        for tx in large_txs:
                            print(f"      ${tx['value_usd']:.2f} - {tx['hash'][:20]}...")
                else:
                    print(f"   ❌ 이 블록에는 ETH 전송 거래가 없습니다 (모두 컨트랙트 호출)")
        
        return True
        
    except Exception as e:
        print(f"❌ 블록 데이터 검증 실패: {e}")
        return False

# 테스트 2: 여러 블록에서 대형 거래 찾기
async def test_multi_block_whale_search():
    """여러 블록에서 대형 거래 심층 검색"""
    print("\n🔍 테스트 2: 여러 블록에서 대형 거래 심층 검색")
    print("-" * 50)
    
    try:
        eth_price = await get_current_crypto_price()
        print(f"💰 현재 ETH 가격: ${eth_price:.2f}")
        
        # 최신 블록 번호
        url = f"https://api.etherscan.io/api?module=proxy&action=eth_blockNumber&apikey={ETHERSCAN_API_KEY}"
        response = await make_api_request(url)
        latest_block = int(response['result'], 16)
        
        all_large_txs = []
        total_blocks_checked = 0
        total_transactions_checked = 0
        
        # 최근 20개 블록 검색
        print(f"🔍 최근 20개 블록에서 1천 달러 이상 거래 검색...")
        
        for i in range(20):
            block_num = latest_block - i
            
            try:
                block_url = f"https://api.etherscan.io/api?module=proxy&action=eth_getBlockByNumber&tag=0x{block_num:x}&boolean=true&apikey={ETHERSCAN_API_KEY}"
                block_data = await make_api_request(block_url)
                
                if not block_data or 'result' not in block_data:
                    continue
                
                block = block_data['result']
                transactions = block.get('transactions', [])
                total_blocks_checked += 1
                total_transactions_checked += len(transactions)
                
                block_large_txs = []
                
                for tx in transactions:
                    try:
                        value_wei = tx.get('value', '0x0')
                        if value_wei == '0x0':
                            continue
                        
                        value_eth = wei_to_ether(value_wei)
                        value_usd = value_eth * eth_price
                        
                        # 1천 달러 이상 거래 수집 (더 낮은 임계값)
                        if value_usd >= 1000:
                            block_large_txs.append({
                                'block': block_num,
                                'hash': tx.get('hash'),
                                'from': tx.get('from'),
                                'to': tx.get('to'),
                                'value_eth': round(value_eth, 6),
                                'value_usd': round(value_usd, 2),
                                'timestamp': int(block.get('timestamp', '0x0'), 16)
                            })
                    
                    except Exception as tx_error:
                        continue
                
                all_large_txs.extend(block_large_txs)
                
                if block_large_txs:
                    print(f"   📦 블록 {block_num}: {len(block_large_txs)}개 대형 거래 발견")
                else:
                    print(f"   📦 블록 {block_num}: 대형 거래 없음 ({len(transactions)}개 거래 중)")
                
                await asyncio.sleep(0.1)  # Rate limit
                
            except Exception as block_error:
                print(f"   ❌ 블록 {block_num} 처리 실패: {block_error}")
                continue
        
        # 결과 분석
        print(f"\n📊 검색 결과 요약:")
        print(f"   🔍 검색한 블록 수: {total_blocks_checked}")
        print(f"   🔍 검색한 총 거래 수: {total_transactions_checked:,}")
        print(f"   🐋 발견된 1천달러+ 거래: {len(all_large_txs)}개")
        
        if all_large_txs:
            # 가장 큰 거래들 표시
            all_large_txs.sort(key=lambda x: x['value_usd'], reverse=True)
            
            print(f"\n🏆 TOP 10 대형 거래:")
            for i, tx in enumerate(all_large_txs[:10], 1):
                timestamp = datetime.fromtimestamp(tx['timestamp'])
                print(f"   {i:2d}. ${tx['value_usd']:>12,.2f} ({tx['value_eth']:>8.4f} ETH) - 블록 {tx['block']}")
                print(f"       🕐 {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"       📤 From: {tx['from'][:20]}...")
                print(f"       📥 To: {tx['to'][:20]}...")
            
            # 통계
            total_volume = sum(tx['value_usd'] for tx in all_large_txs)
            avg_value = total_volume / len(all_large_txs)
            
            print(f"\n📈 거래 통계:")
            print(f"   총 거래량: ${total_volume:,.2f}")
            print(f"   평균 거래 금액: ${avg_value:,.2f}")
            print(f"   최대 거래: ${all_large_txs[0]['value_usd']:,.2f}")
            
            # 임계값별 분석
            thresholds = [1000, 5000, 10000, 50000, 100000]
            print(f"\n🎯 임계값별 거래 수:")
            for threshold in thresholds:
                count = len([tx for tx in all_large_txs if tx['value_usd'] >= threshold])
                print(f"   ${threshold:>6,}달러 이상: {count:>3d}개")
        
        else:
            print(f"   ❌ 1천 달러 이상 거래를 찾을 수 없습니다!")
            print(f"   🤔 가능한 원인:")
            print(f"      - 대부분의 거래가 토큰 전송 (ETH value = 0)")
            print(f"      - 컨트랙트 호출 (ETH value = 0)")
            print(f"      - 현재 시간대에 대형 ETH 전송이 적음")
        
        return all_large_txs
        
    except Exception as e:
        print(f"❌ 대형 거래 검색 실패: {e}")
        return []

# 테스트 3: 알려진 거래소 주소 활동 확인
async def test_known_exchange_activity():
    """알려진 거래소 주소의 실제 활동 확인"""
    print("\n🔍 테스트 3: 알려진 거래소 주소 활동 확인")
    print("-" * 50)
    
    # 실제로 활동이 많은 알려진 주소들
    test_addresses = {
        'Binance_Hot_Wallet': '0x3f5CE5FbfE3E9af3971dd833D26bA9b5C936f0bE',
        'Coinbase_6': '0xA9D1e08C7793af67e9d92fe308d5697FB81d3E43',
        'Kraken_7': '0x2910543Af39abA0Cd09dBb2D50200b3E800A63D2',
        'Bitfinex': '0x1151314c646ce4e0efd76d1af4760ae66a9fe30f',  # 비트파이넥스
        'Huobi': '0xdc76CD25977E0a5Ae17155770273aD58648900D3',     # 후오비
    }
    
    for name, address in test_addresses.items():
        print(f"\n🏦 {name} 주소 활동 확인:")
        print(f"   📍 주소: {address}")
        
        try:
            # 최근 거래 내역 조회
            url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&page=1&offset=10&sort=desc&apikey={ETHERSCAN_API_KEY}"
            
            response = await make_api_request(url)
            if not response or 'result' not in response:
                print(f"   ❌ 거래 내역 조회 실패")
                continue
            
            transactions = response['result']
            print(f"   📊 최근 {len(transactions)}개 거래 확인")
            
            if transactions:
                latest_tx = transactions[0]
                tx_time = datetime.fromtimestamp(int(latest_tx.get('timeStamp', '0')))
                time_diff = datetime.now() - tx_time
                
                print(f"   🕐 최근 거래: {tx_time.strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"   ⏰ 마지막 활동: {time_diff.days}일 {time_diff.seconds//3600}시간 전")
                
                # 최근 24시간 내 거래 확인
                recent_txs = []
                cutoff = datetime.now() - timedelta(hours=24)
                
                for tx in transactions:
                    tx_timestamp = int(tx.get('timeStamp', '0'))
                    tx_time = datetime.fromtimestamp(tx_timestamp)
                    
                    if tx_time > cutoff:
                        value_wei = tx.get('value', '0')
                        value_eth = wei_to_ether(value_wei) if value_wei != '0' else 0
                        
                        recent_txs.append({
                            'hash': tx.get('hash'),
                            'value_eth': value_eth,
                            'time': tx_time,
                            'direction': 'IN' if tx.get('to', '').lower() == address.lower() else 'OUT'
                        })
                
                print(f"   📈 최근 24시간 거래: {len(recent_txs)}개")
                
                if recent_txs:
                    eth_price = await get_current_crypto_price()
                    for tx in recent_txs[:3]:  # 최근 3개만 표시
                        value_usd = tx['value_eth'] * eth_price
                        print(f"      {tx['direction']}: ${value_usd:,.2f} ({tx['value_eth']:.4f} ETH)")
                        print(f"         🕐 {tx['time'].strftime('%H:%M:%S')}")
                else:
                    print(f"   ⚠️ 최근 24시간 내 거래 없음")
            else:
                print(f"   ❌ 거래 내역 없음")
        
        except Exception as e:
            print(f"   ❌ 주소 {name} 확인 실패: {e}")
        
        await asyncio.sleep(0.5)  # Rate limit

# 전체 검증 테스트 실행
async def run_detailed_verification():
    """상세 검증 테스트 실행"""
    print("🚀 상세 데이터 검증 및 고래 거래 디버깅 시작")
    print("=" * 60)
    
    try:
        # 1. 실제 블록 데이터 검증
        await test_detailed_block_data()
        await asyncio.sleep(1)
        
        # 2. 대형 거래 심층 검색
        whale_txs = await test_multi_block_whale_search()
        await asyncio.sleep(1)
        
        # 3. 거래소 주소 활동 확인
        await test_known_exchange_activity()
        
        print(f"\n" + "=" * 60)
        print(f"🎯 검증 결과 요약:")
        print(f"✅ API 연결: 정상")
        print(f"✅ 블록 데이터: 실제 데이터 확인")
        print(f"✅ 거래 분석: 정상 작동")
        
        if whale_txs:
            print(f"🐋 대형 거래 감지: {len(whale_txs)}개 발견")
        else:
            print(f"🤔 대형 거래 미감지: 현재 시간대에 실제로 적음")
        
        print(f"\n💡 결론:")
        print(f"   - 도구는 정상적으로 작동합니다")
        print(f"   - 실제 블록체인 데이터를 정확히 분석합니다")
        print(f"   - 대형 거래가 없는 것은 실제 상황일 가능성이 높습니다")
        
    except Exception as e:
        print(f"\n❌ 검증 실행 중 오류: {e}")

print("📝 사용법:")
print("await run_detailed_verification()  # 전체 상세 검증 실행")
print("await test_detailed_block_data()    # 블록 데이터만 검증")
print("await test_multi_block_whale_search()  # 대형 거래 심층 검색")

# 실행 예시
await run_detailed_verification()

🔍 상세 데이터 검증 및 고래 거래 디버깅
📝 사용법:
await run_detailed_verification()  # 전체 상세 검증 실행
await test_detailed_block_data()    # 블록 데이터만 검증
await test_multi_block_whale_search()  # 대형 거래 심층 검색
🚀 상세 데이터 검증 및 고래 거래 디버깅 시작

🔍 테스트 1: 실제 블록 데이터 상세 검증
--------------------------------------------------
📊 최신 블록: 22,693,937

📦 블록 22693937 상세 정보:
   🕐 타임스탬프: 0x684bc94b
   📊 거래 수: 339
   ⛽ Gas 사용: 0x18a87cb
   ⛽ Gas 한도: 0x226ecf7
   🔗 해시: 0xf1bfd600bcf8ffe6f6c726b367242e3024883667f60891eea07f0f9fb8820fa4

🔍 거래 데이터 샘플 검증 (처음 5개):

   거래 1:
      🔗 해시: 0x188e0d90cb14a0b8d0ff003863161b13324656288fcddc5d779b211a1b785abb
      📤 From: 0x710b2c4bdd1d0b7af60a1620061e69cb6c3153f0
      📥 To: 0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0
      💰 Value (Wei): 0x0
      💰 Value: $0 (토큰 전송 또는 컨트랙트 호출)

   거래 2:
      🔗 해시: 0x96a0e80d04ca1193f531b3d395e03e7fe6b3fb4b3389314dbb2dbeb55f08a628
      📤 From: 0x710b2c4bdd1d0b7af60a1620061e69cb6c3153f0
      📥 To: 0x0439e60f02a8900a951603950d8d4527f400c3f1
      💰 Value (Wei): 0x